In [ ]:
#import all required library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
from keras import Sequential
from keras.layers import Dense


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
url = 'https://raw.githubusercontent.com/arpit1012/MLAPLI/master/bankloan.csv'
df = pd.read_csv(url)
df=df.dropna()
df.isna().any()
df=df.drop('Loan_ID',axis=1)
df['LoanAmount']=(df['LoanAmount']*1000).astype(int)
Counter(df['Loan_Status'])
#data is not too much imbalanced but using smote to convert to balanced increases the accurarcy

Counter({'N': 148, 'Y': 332})

In [ ]:
pre_y=df['Loan_Status']
pre_x=df.drop('Loan_Status',axis=1)
#one hot encodering
dm_x=pd.get_dummies(pre_x)
dm_y=pre_y.map(dict(Y=1,N=0))

In [ ]:
#run smote sampling method
#oversampling means increase the minority to create new sample that could match up to the length of majority

smote=SMOTE(ratio='minority')
X1,y=smote.fit_sample(dm_x,dm_y)



In [ ]:
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,0.056895,0.068948,0.204738,0.72973,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.000000,1.000000
1,0.000000,0.242177,0.155628,0.949239,0.72973,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.000000
2,0.666667,0.047458,0.041700,0.250423,0.72973,1.000000,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.000000,0.000000
3,0.000000,0.043958,0.059107,0.187817,0.72973,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.000000
4,0.000000,0.049845,0.068150,0.292724,0.72973,1.000000,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.274519,0.038069,0.076264,0.273315,0.72973,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.823558,0.176442
129,0.000000,0.138565,0.000000,0.396362,0.72973,0.625066,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.374934,0.625066
130,1.000000,0.040198,0.073884,0.192893,0.72973,0.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.000000
131,0.000000,0.051738,0.072435,0.170897,0.72973,1.000000,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.000000,1.000000


In [ ]:
X__=sc.inverse_transform(X_test)
pd.DataFrame(X__)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.000000,4750.000000,2333.000000,130000.000000,360.0,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.000000,1.000000
1,0.000000,19730.000000,5266.000000,570000.000000,360.0,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.000000
2,2.000000,3987.000000,1411.000000,157000.000000,360.0,1.000000,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.000000,0.000000
3,0.000000,3704.000000,2000.000000,120000.000000,360.0,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.000000
4,0.000000,4180.000000,2306.000000,182000.000000,360.0,1.000000,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,0.823558,3227.865830,2580.529809,170529.326743,360.0,1.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.823558,0.176442
129,0.000000,11352.990282,0.000000,243250.131911,360.0,0.625066,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.374934,0.625066
130,3.000000,3400.000000,2500.000000,123000.000000,360.0,0.000000,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.000000,0.000000
131,0.000000,4333.000000,2451.000000,110000.000000,360.0,1.000000,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.000000,1.000000


In [ ]:
print(pd.DataFrame(y_test),pd.DataFrame(y_pred))

     0
0    1
1    0
2    1
3    1
4    1
..  ..
128  0
129  0
130  0
131  0
132  1

[133 rows x 1 columns]             0
0    0.860650
1    0.003209
2    0.463154
3    0.845099
4    0.993078
..        ...
128  0.841651
129  0.012388
130  0.027648
131  0.814763
132  0.002615

[133 rows x 1 columns]


In [ ]:
sc=MinMaxScaler()
X=sc.fit_transform(X1)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [ ]:
classifier=Sequential()
classifier.add(Dense(200,activation='relu',kernel_initializer='random_normal',input_dim=X_test.shape[1]))
classifier.add(Dense(400,activation='relu',kernel_initializer='random_normal'))
classifier.add(Dense(5,activation='relu',kernel_initializer='random_normal'))
classifier.add(Dense(1,activation='sigmoid',kernel_initializer='random_normal'))
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
classifier.fit(X_train,y_train,batch_size=25,epochs=100,verbose=0)
eval_model=classifier.evaluate(X_train,y_train)
eval_model

17/17 [==============================] - 1s 2ms/step - loss: 0.2564 - accuracy: 0.8945


[0.256407767534256, 0.8945385813713074]

In [ ]:
y_pred=classifier.predict(X_test)
y_pred = (y_pred>0.53)

In [ ]:
cm=confusion_matrix(y_test,y_pred)
ax=plt.subplot()
sns.heatmap(cm,annot=True,ax=ax)

ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion matrix')
ax.xaxis.set_ticklabels(['No','Yes'])
ax.yaxis.set_ticklabels(['No','Yes'])

In [ ]:
#k fold

from sklearn.model_selection import StratifiedKFold
kfold=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
cvscores=[]

for train,test in kfold.split(X,y):
  model=Sequential()
  model.add(Dense(200,input_dim=17,activation='relu'))
  model.add(Dense(400,activation='relu'))
  model.add(Dense(4,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

  model.fit(X[train],y[train],epochs=100,verbose=0)

  scores=model.evaluate(X[test],y[test],verbose=0)

  print("%s: %.2f%%" % (model.metrics_names[1],scores[1]*100))
  cvscores.append(scores[1]*100)
print("%.2f%% (+/- %.2f%%" % (np.mean(cvscores),np.std(cvscores)))

accuracy: 78.83%
accuracy: 81.00%
accuracy: 75.57%
78.46% (+/- 2.23%


In [ ]:
#create pickle
import pickle
from sklearn.externals import joblib
filename='loan_model.pkl'
joblib.dump(classifier,filename)

In [ ]:
from keras.models import load_model

classifier.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'


In [ ]:
sc1 = joblib.load('scalers.pkl')

In [ ]:
import joblib


In [ ]:
sc1

In [ ]:
joblib.dump(sc, 'scaler.pkl') 


In [ ]:
scq=joblib.load('scaler.pkl')